### Title: Video Resizing Script for BD Sports-10 Datasets 

### Step-by-Step Code Description:

**(TensorFlow + OpenCV + tqdm)**

1. **Import Libraries**

   * `os` → Handles file paths and directories.
   * `cv2` (OpenCV) → Reads and writes video files frame by frame.
   * `numpy` → Handles array operations.
   * `tensorflow` → Uses `tf.image.resize_with_pad` to resize frames with aspect ratio preserved (padding applied).
   * `tqdm.notebook` → Displays progress bars inside Jupyter/Kaggle notebooks.
   * `shutil` → Used for zipping the output folder after processing.

---

2. **Function: `format_frames(frame, output_size=(224, 224))`**

   * Takes a single frame (image) as input.
   * Converts it to `float32` for TensorFlow operations.
   * Uses **`tf.image.resize_with_pad`** to resize to the given size (224×224 by default) with padding.
   * Converts back to `uint8` (so OpenCV can handle it).
   * Returns the processed frame.

---

3. **Function: `resize_videos_in_folder(input_path, output_path, output_size=(224, 224), zip_output=True)`**

   * **Creates output folder** (if it doesn’t exist).
   * **Finds all `.mp4` videos** in the input folder.
   * Prints the total number of video files.

---

4. **Loop through each video file**

   * Opens video with `cv2.VideoCapture`.
   * Reads FPS (frames per second) and total number of frames.
   * Prepares a `cv2.VideoWriter` object to save resized videos in **224×224 resolution**.

---

5. **Frame Processing Loop**

   * Iterates through all frames in the video using a tqdm progress bar.
   * Each frame is read in **BGR format** (OpenCV default).
   * Converts to **RGB**.
   * Resizes with padding using `format_frames`.
   * Converts back to **BGR**.
   * Writes the processed frame to the output video.

---

6. **Finalize Each Video**

   * Releases the input and output video streams.
   * Shows a message confirming the saved file and how many remain.

---

7. **After All Videos Processed**

   * Prints completion message.
   * Optionally zips the entire output folder into a `.zip` file using `shutil.make_archive`.

---

8. **Example Usage**

   * Processes all videos inside:
     `/kaggle/input/bd-sports-10/Dataset/Hari_Vanga` and saves the resized versions to: `/kaggle/working/Hari_Vanga`.
   * Each video is resized to **224×224 with padding**.
   * The final output folder is compressed into a **`.zip` archive** for easy storage and sharing.
   * **Note:** You can change the input and output paths to process different dataset folders or sports classes. For example:

     * Input: `/kaggle/input/bd-sports-10/Dataset/Kabaddi`
     * Output: `/kaggle/working/Kabaddi`



In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm 
import shutil

def format_frames(frame, output_size=(224, 224)):
    """
    Resize with padding to output_size (like tf.image.resize_with_pad).
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    frame = tf.image.convert_image_dtype(frame, tf.uint8)
    return frame.numpy()

def resize_videos_in_folder(input_path, output_path, output_size=(224, 224), zip_output=True):
    """
    Resize all videos in a folder to output_size (with padding), save them,
    and optionally zip the output folder.
    """
    os.makedirs(output_path, exist_ok=True)
    video_files = [f for f in os.listdir(input_path) if f.endswith(".mp4")]

    total_files = len(video_files)
    print(f"📂 Total video files found: {total_files}")

   
    for idx, filename in enumerate(tqdm(video_files, desc="Processing videos", unit="file", colour="green")):
        input_video_path = os.path.join(input_path, filename)
        output_video_path = os.path.join(output_path, filename)

        cap = cv2.VideoCapture(input_video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Video writer (final output will always be 224x224)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, output_size)

        
        for _ in tqdm(range(total_frames), desc=f"Frames {filename}", unit="frame", leave=False, colour="green"):
            ret, frame = cap.read()
            if not ret:
                break

            # OpenCV loads as BGR, convert to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Apply resize with padding
            frame_resized = format_frames(frame_rgb, output_size)
            # Convert back to BGR for OpenCV VideoWriter
            frame_bgr = cv2.cvtColor(frame_resized, cv2.COLOR_RGB2BGR)

            out.write(frame_bgr)

        cap.release()
        out.release()

        remaining = total_files - (idx + 1)
        tqdm.write(f"✅ Saved: {output_video_path} | Remaining files: {remaining}")

    print("🎉 All videos resized and saved.")

    # Zip the folder if requested
    if zip_output:
        zip_path = f"{output_path}.zip"
        shutil.make_archive(zip_path.replace('.zip', ''), 'zip', output_path)
        print(f"📦 Zipped folder created at: {zip_path}")

# Example usage
resize_videos_in_folder(
    input_path="/kaggle/input/bd-sports-10/Dataset/Hari_Vanga",
    output_path="/kaggle/working/Hari_Vanga",
    output_size=(224, 224),
    zip_output=True
)
